<a href="https://colab.research.google.com/github/SeongwonTak/TIL_swtak/blob/master/DLBasic/IntroDLwithPytorch_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 자연어 전처리의 기본 이해

## 토큰화
주어진 텍스트를 단어, 문자 단위로 자르는 것

In [ ]:
en_text = "The quick brown fox jumps over the lazy dog"

In [ ]:
import spacy
spacy_en = spacy.load('en')

In [ ]:
def tokenize(en_text):
  return [tok.text for tok in spacy_en.tokenizer(en_text)]

In [ ]:
print(tokenize(en_text))

['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']


In [ ]:
#  다른 방법 : NLTK 사용하기
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
print(word_tokenize(en_text))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']


한국어의 경우는 을, 를, 은, 는 등의 조사 접사 등의 문제가 있어 토큰화를 띄어쓰기 단어대로 할 경우 조사가 붙어 다른 단어로 인식될 수 있어 형태소 토큰화를 사용한다.

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.5MB/s 
     |████████████████████████████████| 460kB 33.6MB/s 
     |████████████████████████████████| 92kB 6.7MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-05-13 14:20:37--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c2:513, 2406:da00:ff00::6b17:d1f5, ...
Conn

In [ ]:
kor_text = "사과를 먹지 못해서 사과해야 하는데 오늘 사과들을 먹으려고 했는데 사과가 썩어서 슈퍼에 가서 다시 사과를 사서 사과했어."

In [ ]:
from konlpy.tag import Mecab
tokenizer = Mecab()
print(tokenizer.morphs(kor_text))

['사과', '를', '먹', '지', '못해서', '사과', '해야', '하', '는데', '오늘', '사과', '들', '을', '먹', '으려고', '했', '는데', '사과', '가', '썩', '어서', '슈퍼', '에', '가', '서', '다시', '사과', '를', '사', '서', '사과', '했', '어', '.']


## 단어 집합의 생성

중복을 제거한 텍스트의 총 단어 집합을 의미한다.

In [3]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 10.3MB/s 
     |████████████████████████████████| 460kB 36.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [9]:
import urllib.request
import pandas as pd
from konlpy.tag import Mecab
from nltk import FreqDist
import numpy as np
import matplotlib.pyplot as plt

In [5]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
data = pd.read_table('ratings.txt') # 데이터프레임에 저장
data[:10]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
5,2190435,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1
6,9279041,완전 감동입니다 다시봐도 감동,1
7,7865729,개들의 전쟁2 나오나요? 나오면 1빠로 보고 싶음,1
8,7477618,굿,1
9,9250537,바보가 아니라 병 쉰 인듯,1


In [6]:
sample_data = data[:100]

In [7]:
sample_data['document'] = sample_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# 한글과 공백을 제외하고 모두 제거
sample_data[:10]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
5,2190435,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1
6,9279041,완전 감동입니다 다시봐도 감동,1
7,7865729,개들의 전쟁 나오나요 나오면 빠로 보고 싶음,1
8,7477618,굿,1
9,9250537,바보가 아니라 병 쉰 인듯,1


In [ ]:
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
# 형태소 분석 :Mecab
tokenizer = Mecab()
tokenized=[]
for sentence in sample_data['document']:
    temp = tokenizer.morphs(sentence) # 토큰화
    temp = [word for word in temp if not word in stopwords] # 불용어 제거
    tokenized.append(temp)
print(tokenized[:3])

## TorchText

파이토치 자체 기능.

In [12]:
pip install torchtext

### 영어 다루기

In [13]:
import urllib.request
import pandas as pd

In [14]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1')
df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


실습 데이터는, IMDB 데이터로, 데이터가 긍정인지, 부정인지 나타낸다.
1은 긍정, 0은 부정을 나타낸다.

In [15]:
print(len(df))

50000


In [16]:
train_df = df[:25000]
test_df = df[25000:]
# 각각을 나눠 파일로 저장한다.
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

#### 처리 순서
1. 필드 정의하기
2. 데이터셋 만들기
3. 단어 집합 만들기
4. 데이터로더 만들기


#### 필드 정의하기

In [21]:
!pip install torchtext

In [ ]:
from torchtext import data # torchtext.data 임포트

# 필드 정의
TEXT = data.Field(sequential=True, # 시퀀스 데이터인가?
                  use_vocab=True, # 단어 집합을 만들 것인가?
                  tokenize=str.split, # 사용할 토큰화함수
                  lower=True, # 영어 데이터 전부 소문자화
                  batch_first=True, # 미니배치 자차원을 맨 앞으로 하여 데이터를 불러올 것인가.
                  fix_length=20) # 최대 허용 길이

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

#### 데이터셋 만들기

In [ ]:
from torchtext.data import TabularDataset
train_data, test_data = TabularDataset.splits(
        path='.', train='train_data.csv', test='test_data.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

#### 단어집합 만들기

In [ ]:
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)

### 한국어다루기
한국어의 경우는 tokenizer가 바뀐다,